In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls drive/My\ Drive/zindi/Spectrograms_1_Train | wc -l

1857


In [ ]:
!ls drive/My\ Drive/zindi

01.first_try_with_splitted_spec.ipynb
01.spectrograms_all_loop1.h5
01.spectrograms_all_loop2.h5
01.spectrograms_first_try_batch_size_32_loop.h5
02.spec_submission.ipynb
error.txt
Graph
references
SampleSubmission.csv
Spectrograms_1_Train
Spectrograms_2_Test
Spectrograms_all_classified
Spectrograms_splitted_classified
Test
Test.csv
Train
Train.csv
x_test.csv
x_train.csv


In [ ]:
import pandas as pd
import numpy as np

import IPython.display as ipd
from matplotlib import pyplot as plt
import seaborn as sns

import librosa # package for music and audio processing, & features extraction 
import os, shutil, glob

In [ ]:
sub = pd.read_csv('drive/My Drive/zindi/SampleSubmission.csv')

# retrieve all the class names in a list (the 1st col is the id)
birds = sub.columns[1:]

# add a col with all files' paths 
sub['file_path'] = 'drive/My Drive/zindi/Test/' + sub['ID'] + '.mp3'
sub.head()

,ID,Ring-necked Dove,Black Cuckoo,Red-chested Cuckoo,Fiery-necked Nightjar,Green Wood Hoopoe,Crested Barbet,Cape Batis,Olive Bushshrike,Orange-breasted Bushshrike,Bokmakierie,Black-backed Puffback,Southern Boubou,Black-headed Oriole,Fork-tailed Drongo,African Paradise Flycatcher,Sabota Lark,Eastern Clapper Lark,Rufous-naped Lark,Dark-capped Bulbul,Sombre Greenbul,Long-billed Crombec,African Reed Warbler,Rattling Cisticola,Levaillants Cisticola,Tawny-flanked Prinia,Karoo Prinia,Bar-throated Apalis,Green-backed Camaroptera,Chestnut-vented Warbler,Cape White-eye,Karoo Scrub Robin,White-browed Scrub Robin,Cape Robin-Chat,White-browed Robin-Chat,Chorister Robin-Chat,Southern Double-collared Sunbird,White-bellied Sunbird,African Pipit,African Rock Pipit,Cape Bunting,file_path
0,019OYB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/019OYB.mp3
1,01S9OX,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/01S9OX.mp3
2,02CS12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/02CS12.mp3
3,02LM3W,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/02LM3W.mp3
4,0C3A2V,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,drive/My Drive/zindi/Test/0C3A2V.mp3


In [ ]:
# check if ID are alphabetically ordered
col_id = list(sub['ID'].values)
col_id == sorted(list(sub['ID'].values))

True

In [ ]:
sub['len'] = sub['ID'].apply(lambda x: len(x))
sub['len'].value_counts()

6    911
Name: len, dtype: int64

In [ ]:
# same thing with train
train = pd.read_csv('drive/My Drive/zindi/Train.csv')
train['file_path'] = 'drive/My Drive/zindi/Train/' + train['ID'] + '.mp3'
train.head()

,ID,common_name,file_path
0,MBMG2C,Ring-necked Dove,drive/My Drive/zindi/Train/MBMG2C.mp3
1,K8LJSB,Ring-necked Dove,drive/My Drive/zindi/Train/K8LJSB.mp3
2,OGD9L6,Ring-necked Dove,drive/My Drive/zindi/Train/OGD9L6.mp3
3,581PCQ,Ring-necked Dove,drive/My Drive/zindi/Train/581PCQ.mp3
4,P91M1F,Ring-necked Dove,drive/My Drive/zindi/Train/P91M1F.mp3


In [ ]:
nb_class = len(birds)
nb_class

40

In [ ]:
input_shape = (512, 512, 3)

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Flatten, Dense
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
def create_model():
    model = Sequential()
    # Layer C1
    model.add(Conv2D(filters=6, kernel_size=(3, 3), padding='same', activation='relu', input_shape=input_shape))
    # Layer S2
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    # Layer C3
    model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='same', activation='relu'))
    # Layer S4
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    # Before going into layer C5, we flatten our units
    model.add(Flatten())
    # Layer C5
    model.add(Dense(units=512, activation='relu'))
    # Layer F6
    model.add(Dense(units=128, activation='relu'))
    # Output layer
    model.add(Dense(units=nb_class, activation='softmax')) # softmax for multi class
    return model



In [ ]:
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the callbacks
callbacks = [EarlyStopping(monitor='val_loss', patience=5), TensorBoard(log_dir='drive/My Drive/zindi/Graph', histogram_freq=0, write_graph=True, write_images=True)]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
# https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

train_datagen = ImageDataGenerator(
        rescale=1./255,           # normalization
        rotation_range=45,        # a value in degrees (0-180), a range within which to randomly rotate pictures
        width_shift_range=0.2,    # a fraction of total width or height, within which to randomly translate pictures vertically or horizontally
        height_shift_range=0.2,
        shear_range=0.2,          # for randomly applying shearing transformations
        zoom_range=0.2,           # for randomly zooming inside pictures
        horizontal_flip=False,    # for randomly flipping half of the images horizontally --relevant when there are no assumptions of horizontal assymetry
        fill_mode='nearest')      # used for filling in newly created pixels, which can appear after a rotation or a width/height shift

batch_size = 32
target_size=(512, 512)

train_generator = train_datagen.flow_from_directory(
        directory='drive/My Drive/zindi/Spectrograms_all_classified',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical') # for multi class

Found 1857 images belonging to 40 classes.


In [ ]:
for i in range(1, 6):
    model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=40,
        #validation_data=validation_generator,
        #validation_steps=800 // batch_size,
        callbacks=callbacks)

    # Save the model's weights
    model.save(f'drive/My Drive/zindi/01.spectrograms_all_loop{i}.h5')
    print(f"saved model #{i}")

Epoch 1/40
62/62 [==============================] - 718s 12s/step - loss: 4.1933 - acc: 0.0538
Epoch 2/40
62/62 [==============================] - 385s 6s/step - loss: 3.5628 - acc: 0.0573
Epoch 3/40
62/62 [==============================] - 377s 6s/step - loss: 3.5356 - acc: 0.0624
Epoch 4/40
62/62 [==============================] - 381s 6s/step - loss: 3.5066 - acc: 0.0666
Epoch 5/40
62/62 [==============================] - 388s 6s/step - loss: 3.5209 - acc: 0.0756
Epoch 6/40
62/62 [==============================] - 383s 6s/step - loss: 3.5509 - acc: 0.0707
Epoch 7/40
62/62 [==============================] - 382s 6s/step - loss: 3.4755 - acc: 0.0937
Epoch 8/40
62/62 [==============================] - 382s 6s/step - loss: 3.4523 - acc: 0.0876
Epoch 9/40
62/62 [==============================] - 376s 6s/step - loss: 3.4424 - acc: 0.0900
Epoch 10/40
62/62 [==============================] - 388s 6s/step - loss: 3.4052 - acc: 0.0912
Epoch 11/40
62/62 [==============================] - 378s 

KeyboardInterrupt: ignored

In [ ]:
model.load_weights('drive/My Drive/zindi/01.spectrograms_all_loop2.h5')

In [ ]:
!ls drive/My\ Drive/zindi/Spectrograms_2_Test/Spectrograms_2_Test | wc -l

911


In [ ]:
# There are no labelled test images. In this case, you will have a single test folder which contains all the images that you want to classify.

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
        directory='drive/My Drive/zindi/Spectrograms_2_Test/Spectrograms_2_Test',
        classes=['test'],
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',  # our generator will only yield batches of data, no labels
        shuffle=False)  # preserve the order of filenames and predictions

# https://kylewbanks.com/blog/loading-unlabeled-images-with-imagedatagenerator-flowfromdirectory-keras

Found 0 images belonging to 1 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
        directory='drive/My Drive/zindi/Spectrograms_2_Test/',
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False
        )

Found 911 images belonging to 1 classes.


In [ ]:
test_generator.reset()

# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
probabilities = model.predict_generator(generator=test_generator, steps=911 // batch_size + 1)

In [ ]:
probabilities

array([[9.4093883e-04, 2.6373730e-09, 6.6080523e-11, ..., 2.0905470e-10,
        2.6763013e-05, 1.8598750e-03],
       [5.4241408e-04, 6.0556936e-03, 1.1632715e-02, ..., 1.5241872e-04,
        4.3900189e-05, 9.1250956e-02],
       [1.0583223e-03, 5.4447481e-04, 6.1137779e-03, ..., 5.3610040e-07,
        4.1149196e-05, 4.2756028e-03],
       ...,
       [7.0689120e-03, 5.0955052e-05, 2.1888958e-03, ..., 6.4265901e-06,
        1.4341922e-02, 4.8357803e-02],
       [1.1564338e-03, 5.4084994e-03, 5.1313377e-04, ..., 1.8568212e-03,
        7.9082449e-05, 7.9810908e-03],
       [8.8576468e-05, 1.1749296e-02, 1.5668349e-05, ..., 3.1056777e-02,
        3.3760255e-06, 5.7121946e-07]], dtype=float32)

In [ ]:
labels = (train_generator.class_indices)

# reorder class labels according to the indices
labels = dict((v,k) for k,v in labels.items())

list(labels.values())

['African Paradise Flycatcher',
 'African Pipit',
 'African Reed Warbler',
 'African Rock Pipit',
 'Bar-throated Apalis',
 'Black Cuckoo',
 'Black-backed Puffback',
 'Black-headed Oriole',
 'Bokmakierie',
 'Cape Batis',
 'Cape Bunting',
 'Cape Robin-Chat',
 'Cape White-eye',
 'Chestnut-vented Warbler',
 'Chorister Robin-Chat',
 'Crested Barbet',
 'Dark-capped Bulbul',
 'Eastern Clapper Lark',
 'Fiery-necked Nightjar',
 'Fork-tailed Drongo',
 'Green Wood Hoopoe',
 'Green-backed Camaroptera',
 'Karoo Prinia',
 'Karoo Scrub Robin',
 'Levaillants Cisticola',
 'Long-billed Crombec',
 'Olive Bushshrike',
 'Orange-breasted Bushshrike',
 'Rattling Cisticola',
 'Red-chested Cuckoo',
 'Ring-necked Dove',
 'Rufous-naped Lark',
 'Sabota Lark',
 'Sombre Greenbul',
 'Southern Boubou',
 'Southern Double-collared Sunbird',
 'Tawny-flanked Prinia',
 'White-bellied Sunbird',
 'White-browed Robin-Chat',
 'White-browed Scrub Robin']

In [ ]:
pred = pd.DataFrame(probabilities, columns=list(labels.values()))

# Get filenames (set shuffle=false in generator is important)
filenames = test_generator.filenames

pred['ID'] = [file_path[20:-4] for file_path in filenames]
pred.head()

,African Paradise Flycatcher,African Pipit,African Reed Warbler,African Rock Pipit,Bar-throated Apalis,Black Cuckoo,Black-backed Puffback,Black-headed Oriole,Bokmakierie,Cape Batis,Cape Bunting,Cape Robin-Chat,Cape White-eye,Chestnut-vented Warbler,Chorister Robin-Chat,Crested Barbet,Dark-capped Bulbul,Eastern Clapper Lark,Fiery-necked Nightjar,Fork-tailed Drongo,Green Wood Hoopoe,Green-backed Camaroptera,Karoo Prinia,Karoo Scrub Robin,Levaillants Cisticola,Long-billed Crombec,Olive Bushshrike,Orange-breasted Bushshrike,Rattling Cisticola,Red-chested Cuckoo,Ring-necked Dove,Rufous-naped Lark,Sabota Lark,Sombre Greenbul,Southern Boubou,Southern Double-collared Sunbird,Tawny-flanked Prinia,White-bellied Sunbird,White-browed Robin-Chat,White-browed Scrub Robin,ID
0,0.000941,2.637373e-09,6.608052e-11,0.000027,7.598268e-07,0.001853,0.000009,0.018101,0.455358,0.004731,0.000004,0.000003,8.623413e-10,0.000680,0.032926,8.139167e-07,0.002353,0.000163,0.418655,0.000279,5.735241e-08,9.965642e-07,9.437827e-09,3.538213e-09,0.000003,0.000004,0.003984,0.003706,0.000001,0.043004,0.005414,0.001519,4.141566e-08,0.001522,0.002872,2.363436e-09,1.843040e-11,2.090547e-10,0.000027,0.001860,019OYB
1,0.000542,6.055694e-03,1.163272e-02,0.018021,4.457813e-02,0.000644,0.003124,0.007036,0.102694,0.001707,0.023632,0.064323,2.390646e-02,0.271880,0.000598,2.422623e-03,0.013938,0.049346,0.000282,0.004358,5.324751e-04,1.856168e-02,1.995381e-03,5.040558e-03,0.086465,0.070621,0.000762,0.001162,0.038342,0.000144,0.008382,0.005292,1.219494e-02,0.007492,0.000327,7.515202e-06,5.109336e-04,1.524187e-04,0.000044,0.091251,01S9OX
2,0.001058,5.444748e-04,6.113778e-03,0.005678,4.373532e-03,0.003307,0.000496,0.007918,0.078121,0.002996,0.014161,0.027945,6.380689e-04,0.009394,0.008007,2.441104e-03,0.011783,0.469785,0.044146,0.003394,3.499550e-02,1.757451e-05,5.927449e-04,9.713661e-04,0.011200,0.004240,0.009998,0.000134,0.005094,0.000019,0.195565,0.007659,1.846615e-02,0.000147,0.004267,1.369243e-05,7.374890e-07,5.361004e-07,0.000041,0.004276,02CS12
3,0.007521,8.113311e-04,2.086589e-03,0.001858,7.156504e-02,0.009691,0.006238,0.006893,0.159682,0.024400,0.005056,0.017723,2.275281e-02,0.049102,0.026741,1.629203e-02,0.027028,0.019961,0.034430,0.002862,4.101929e-03,2.171398e-02,6.809002e-04,1.061245e-03,0.060673,0.011063,0.036813,0.024352,0.018940,0.004902,0.200061,0.012166,1.661147e-03,0.005723,0.014561,4.254184e-05,1.449387e-04,6.221540e-05,0.001457,0.067123,02LM3W
4,0.012737,1.164183e-03,3.695672e-02,0.000203,3.182721e-02,0.000706,0.132442,0.000813,0.006435,0.001534,0.000887,0.013616,2.178050e-02,0.005636,0.015150,1.267102e-02,0.048848,0.000422,0.000039,0.215401,3.221766e-02,1.270064e-01,9.602748e-04,9.495996e-04,0.032088,0.007399,0.004912,0.000771,0.099187,0.004062,0.000766,0.000106,1.887102e-02,0.006280,0.069220,1.103027e-03,5.418082e-03,1.917057e-02,0.005523,0.004721,0C3A2V


In [ ]:
pred.shape

(911, 41)

In [ ]:
sub.columns

Index(['ID', 'Ring-necked Dove', 'Black Cuckoo', 'Red-chested Cuckoo',
       'Fiery-necked Nightjar', 'Green Wood Hoopoe', 'Crested Barbet',
       'Cape Batis', 'Olive Bushshrike', 'Orange-breasted Bushshrike',
       'Bokmakierie', 'Black-backed Puffback', 'Southern Boubou',
       'Black-headed Oriole', 'Fork-tailed Drongo',
       'African Paradise Flycatcher', 'Sabota Lark', 'Eastern Clapper Lark',
       'Rufous-naped Lark', 'Dark-capped Bulbul', 'Sombre Greenbul',
       'Long-billed Crombec', 'African Reed Warbler', 'Rattling Cisticola',
       'Levaillants Cisticola', 'Tawny-flanked Prinia', 'Karoo Prinia',
       'Bar-throated Apalis', 'Green-backed Camaroptera',
       'Chestnut-vented Warbler', 'Cape White-eye', 'Karoo Scrub Robin',
       'White-browed Scrub Robin', 'Cape Robin-Chat',
       'White-browed Robin-Chat', 'Chorister Robin-Chat',
       'Southern Double-collared Sunbird', 'White-bellied Sunbird',
       'African Pipit', 'African Rock Pipit', 'Cape Bunting', 'f

In [ ]:
try:
    sub = sub.drop(columns=['len', 'file_path'])
except:
    pass

pred = pred[list(sub.columns)]
pred.head()

,ID,Ring-necked Dove,Black Cuckoo,Red-chested Cuckoo,Fiery-necked Nightjar,Green Wood Hoopoe,Crested Barbet,Cape Batis,Olive Bushshrike,Orange-breasted Bushshrike,Bokmakierie,Black-backed Puffback,Southern Boubou,Black-headed Oriole,Fork-tailed Drongo,African Paradise Flycatcher,Sabota Lark,Eastern Clapper Lark,Rufous-naped Lark,Dark-capped Bulbul,Sombre Greenbul,Long-billed Crombec,African Reed Warbler,Rattling Cisticola,Levaillants Cisticola,Tawny-flanked Prinia,Karoo Prinia,Bar-throated Apalis,Green-backed Camaroptera,Chestnut-vented Warbler,Cape White-eye,Karoo Scrub Robin,White-browed Scrub Robin,Cape Robin-Chat,White-browed Robin-Chat,Chorister Robin-Chat,Southern Double-collared Sunbird,White-bellied Sunbird,African Pipit,African Rock Pipit,Cape Bunting
0,019OYB,0.005414,0.001853,0.043004,0.418655,5.735241e-08,8.139167e-07,0.004731,0.003984,0.003706,0.455358,0.000009,0.002872,0.018101,0.000279,0.000941,4.141566e-08,0.000163,0.001519,0.002353,0.001522,0.000004,6.608052e-11,0.000001,0.000003,1.843040e-11,9.437827e-09,7.598268e-07,9.965642e-07,0.000680,8.623413e-10,3.538213e-09,0.001860,0.000003,0.000027,0.032926,2.363436e-09,2.090547e-10,2.637373e-09,0.000027,0.000004
1,01S9OX,0.008382,0.000644,0.000144,0.000282,5.324751e-04,2.422623e-03,0.001707,0.000762,0.001162,0.102694,0.003124,0.000327,0.007036,0.004358,0.000542,1.219494e-02,0.049346,0.005292,0.013938,0.007492,0.070621,1.163272e-02,0.038342,0.086465,5.109336e-04,1.995381e-03,4.457813e-02,1.856168e-02,0.271880,2.390646e-02,5.040558e-03,0.091251,0.064323,0.000044,0.000598,7.515202e-06,1.524187e-04,6.055694e-03,0.018021,0.023632
2,02CS12,0.195565,0.003307,0.000019,0.044146,3.499550e-02,2.441104e-03,0.002996,0.009998,0.000134,0.078121,0.000496,0.004267,0.007918,0.003394,0.001058,1.846615e-02,0.469785,0.007659,0.011783,0.000147,0.004240,6.113778e-03,0.005094,0.011200,7.374890e-07,5.927449e-04,4.373532e-03,1.757451e-05,0.009394,6.380689e-04,9.713661e-04,0.004276,0.027945,0.000041,0.008007,1.369243e-05,5.361004e-07,5.444748e-04,0.005678,0.014161
3,02LM3W,0.200061,0.009691,0.004902,0.034430,4.101929e-03,1.629203e-02,0.024400,0.036813,0.024352,0.159682,0.006238,0.014561,0.006893,0.002862,0.007521,1.661147e-03,0.019961,0.012166,0.027028,0.005723,0.011063,2.086589e-03,0.018940,0.060673,1.449387e-04,6.809002e-04,7.156504e-02,2.171398e-02,0.049102,2.275281e-02,1.061245e-03,0.067123,0.017723,0.001457,0.026741,4.254184e-05,6.221540e-05,8.113311e-04,0.001858,0.005056
4,0C3A2V,0.000766,0.000706,0.004062,0.000039,3.221766e-02,1.267102e-02,0.001534,0.004912,0.000771,0.006435,0.132442,0.069220,0.000813,0.215401,0.012737,1.887102e-02,0.000422,0.000106,0.048848,0.006280,0.007399,3.695672e-02,0.099187,0.032088,5.418082e-03,9.602748e-04,3.182721e-02,1.270064e-01,0.005636,2.178050e-02,9.495996e-04,0.004721,0.013616,0.005523,0.015150,1.103027e-03,1.917057e-02,1.164183e-03,0.000203,0.000887


In [ ]:
# check if ID are alphabetically ordered
col_id = list(pred['ID'].values)
col_id == sorted(list(pred['ID'].values))

True

In [ ]:
pred.to_csv('drive/My Drive/zindi/first_sub.csv', index=False)

submission score 3.2652909169116

In [ ]:
3.2652909169116
11
1


In [ ]:
print('hello')

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = np.array([0] * 1000 + [1] * 1000)
y_pred = probabilities > 0.5

confusion_matrix(y_true, y_pred)

In [ ]:
from keras.utils import to_categorical

y_pred_train = to_categorical(model.predict(X_train_norm).argmax(axis=1), num_classes=10)
y_pred_test = to_categorical(model.predict(X_test_norm).argmax(axis=1), num_classes=10)

In [ ]:
# img processing
load_ = []
for n in os.listdir(filepath + "/images/"):
    load_.append(img_to_array(load_img(filepath + "/images/" + n, target_size=(350, 350, 1), grayscale=True)))

X = np.array(load_)
X = X/255



In [ ]:
# Recreate the exact same model purely from the file
# new_model = keras.models.load_model('path_to_my_model.h5')

Visualize training results

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification?hl=da

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

KeyError: ignored